In [1]:
import pyodbc
import pandas as pd

server = 'DESKTOP-QT7SBR5'  
database = 'imdb'  
username = 'sa'  
password = '12345' 

conn = pyodbc.connect(
    f'DRIVER={{SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)
query = """
select f.Films_ID, 
       f.Title, 

       -- Loại bỏ trùng lặp cho Genres
       stuff((
           select distinct ', ' + g.Genres
           from FilmsGenres fg
           join Genres g on fg.Genre_ID = g.Genre_ID
           where fg.Films_ID = f.Films_ID
           for xml path(''), type).value('.', 'nvarchar(max)'), 1, 2, '') as Genres,
       
       -- Loại bỏ trùng lặp cho Stars
       stuff((
           select distinct ', ' + s.Stars
           from FilmsStars fs
           join Stars s on fs.Star_ID = s.Star_ID
           where fs.Films_ID = f.Films_ID
           for xml path(''), type).value('.', 'nvarchar(max)'), 1, 2, '') as Stars,

		 f.Plot_Summary

from Films f
group by f.Films_ID, f.Title, f.Plot_Summary
order by f.Films_ID asc;

"""

movies_df= pd.read_sql_query(query, conn)
conn.close()

movies_df.head()

C:\Users\Welcome\AppData\Local\Temp\ipykernel_11956\3894926705.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies_df= pd.read_sql_query(query, conn)


,Films_ID,Title,Genres,Stars,Plot_Summary
0,1,Phineas and Ferb the Movie: Candace Against th...,"Adventure, Animation, Comedy","Ashley Tisdale, Dee Bradley Baker, Vincent Mar...",The famed stepbrother inventors know what they...
1,2,Toby Goes to Camp,Family,"Adan Allende, Ben F. Campbell, Laura Bilgeri","One part animal adventure, one part human musi..."
2,3,Puff: Wonders of the Reef,"Documentary, Family",Rose Byrne,A baby pufferfish travels through a wondrous m...
3,4,Hockney,Documentary,"Arthur Lambert, Colin Self, David Hockney",The film looks back at Hockney's formative yea...
4,5,Andrew Santino: Cheeseburger,"Comedy, Documentary",Unknown,No topic is safe in this unfiltered stand-up s...


In [2]:
movies_df['Title'] = movies_df['Title'].fillna('')
movies_df['Genres'] = movies_df['Genres'].fillna('')
movies_df['Stars'] = movies_df['Stars'].fillna('')

movies_df['tfidf_features'] = (
    movies_df['Title'] + ' ' +
    movies_df['Genres'] + ' ' +
    movies_df['Stars'] 
)

movies_df['tfidf_features']

0        Phineas and Ferb the Movie: Candace Against th...
1        Toby Goes to Camp Family Adan Allende, Ben F. ...
2        Puff: Wonders of the Reef Documentary, Family ...
3        Hockney Documentary Arthur Lambert, Colin Self...
4        Andrew Santino: Cheeseburger Comedy, Documenta...
                               ...                        
10214    The SML YTP Movie 2: Dimension Trouble Action ...
10215    Food Choices Documentary Joe Cross, Pam Popper...
10216    20 Days in Mariupol Documentary, War Evgeniy M...
10217    I See You Comedy, Drama, Romance Alessandra De...
10218    Fireball: Visitors from Darker Worlds Document...
Name: tfidf_features, Length: 10219, dtype: object

In [3]:
print(movies_df['tfidf_features'].isna().sum())


0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa bằng TF-IDF (Title, Genres, Stars, Director)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['tfidf_features'])

In [5]:
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

# Encode các Plot_Summary
plot_embeddings = bert_model.encode(movies_df['Plot_Summary'].tolist(), batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/320 [00:00<?, ?it/s]

In [6]:
# Kết hợp TF-IDF và BERT embeddings thành một ma trận đặc trưng chung để làm cơ sở cho việc tính toán độ tương đồng
tfidf_weight = 0.5
bert_weight = 0.5
combined_features = np.hstack([tfidf_matrix.toarray() * tfidf_weight, plot_embeddings * bert_weight])

In [32]:
def multi_condition_recommendation(query_dict, movies_data, tfidf_vectorizer, bert_model, combined_features, top_n=5):
    """
    Gợi ý phim dựa trên một hoặc nhiều điều kiện

    Parameters:
        query_dict (dict): Dictionary chứa điều kiện query, e.g., {"Genres": "Action", "Stars": "Tom Hanks"}.
        movies_data (DataFrame): Tập dữ liệu phim.
        tfidf_vectorizer (TfidfVectorizer): TF-IDF vectorizer cho các trường.
        bert_model (SentenceTransformer): Mô hình BERT cho Plot_Summary.
        combined_features (ndarray): Ma trận đặc trưng kết hợp TF-IDF và BERT.
        top_n (int): Số phim gợi ý.

    Returns:
        list: Danh sách các phim được gợi ý cùng điểm tương đồng.
    """
    valid_keys = ['Title', 'Genres', 'Stars', 'Plot_Summary']  # Danh sách các key hợp lệ
    total_similarity = np.zeros(combined_features.shape[0])
    num_conditions = 0  # Số điều kiện hợp lệ đã được xử lý

    # Tính toán độ tương đồng cho từng điều kiện
    for query_type, query_input in query_dict.items():
        # Chuẩn hóa key: kiểm tra key hợp lệ
        query_type_normalized = query_type.strip().title().replace(" ", "_")
        if query_type_normalized not in valid_keys:
            print(f"Invalid query type: {query_type}. Skipping...")
            continue

        if query_type_normalized == 'Genres':
            # Xử lý thể loại nhiều thể loại (Ví dụ: "Drama, Music")
            genres_list = query_input.split(',')  # Tách các thể loại
            genres_input = " ".join([genre.strip() for genre in genres_list])  # Chuỗi thể loại đã được tách
            query_vector_tfidf = tfidf_vectorizer.transform([genres_input])
            query_vector_combined = np.hstack([query_vector_tfidf.toarray(), np.zeros((1, bert_model.encode([""]).shape[1]))])

        # Tính toán TF-IDF hoặc BERT embedding dựa trên key hợp lệ
        elif query_type_normalized in ['Title', 'Stars']:
            query_vector_tfidf = tfidf_vectorizer.transform([query_input])
            query_vector_combined = np.hstack([query_vector_tfidf.toarray(), np.zeros((1, plot_embeddings.shape[1]))])
            
        elif query_type_normalized == 'Plot_Summary':
            query_vector_bert = bert_model.encode([query_input])
            query_vector_combined = np.hstack([np.zeros((1, tfidf_matrix.shape[1])), query_vector_bert])

        # Tính độ tương đồng và cộng điểm vào tổng
        cosine_sim = cosine_similarity(query_vector_combined, combined_features).flatten()
        total_similarity += cosine_sim
        num_conditions += 1

    # Nếu có ít nhất một điều kiện, chuẩn hóa điểm tổng theo số điều kiện
    if num_conditions > 0:
        total_similarity /= num_conditions

    # Lấy các phim tương tự nhất
    similar_indices = np.argsort(total_similarity)[-top_n - 1:-1][::-1]
    recommendations = [(movies_data.iloc[i]['Title'], total_similarity[i]) for i in similar_indices]
    return recommendations


# Kết quả

In [33]:
query_dict = {
    "Genres": "Action",
    "Stars": "Tom Hanks",
    "Plot_Summary": "A thrilling story of survival in space."
}

recommendations = multi_condition_recommendation(
    query_dict=query_dict,
    movies_data=movies_df,
    tfidf_vectorizer=tfidf_vectorizer,
    bert_model=bert_model,
    combined_features=combined_features,
    top_n=5
)

print("Recommended movies based on multiple conditions:")
for title, score in recommendations:
    print(f"{title}: {score:.2f}")


Recommended movies based on multiple conditions:
Code Duello: From the Tithebanner Saga: 0.21
Elysium: 0.20
Alien: Romulus: 0.20
Planet of the Astronauts: 0.20
Living Universe: 0.20


In [34]:
query_dict = {
    "Genres": "Action, Comedy",
    "Stars": "Ryan Reynolds",
    }

recommendations = multi_condition_recommendation(
    query_dict=query_dict,
    movies_data=movies_df,
    tfidf_vectorizer=tfidf_vectorizer,
    bert_model=bert_model,
    combined_features=combined_features,
    top_n=5
)

print("Recommended movies based on multiple conditions:")
for title, score in recommendations:
    print(f"{title}: {score:.2f}")


Recommended movies based on multiple conditions:
The Adam Project: 0.08
Adagium: 0.08
Free Guy: 0.08
Once Upon a Deadpool: 0.08
The Croods: 0.08


In [36]:
# Tạo query với nhiều điều kiện
query_dict = {
   "Plot_Summary": "A thrilling story of survival in space."
}


# Gợi ý phim dựa trên các điều kiện
recommendations = multi_condition_recommendation(
    query_dict=query_dict,
    movies_data=movies_df,
    tfidf_vectorizer=tfidf_vectorizer,
    bert_model=bert_model,
    combined_features=combined_features,
    top_n=5,
)

# Hiển thị kết quả
print("Recommended movies based on conditions:")
for title, score in recommendations:
    print(f"{title}: {score:.2f}")


Recommended movies based on conditions:
Alien: Romulus: 0.60
Planet of the Astronauts: 0.59
Living Universe: 0.59
Elysium: 0.57
Titina: 0.55


In [35]:
# Tạo query với nhiều điều kiện
query_dict = {
   "Stars": "Tom Hanks"
}


# Gợi ý phim dựa trên các điều kiện
recommendations = multi_condition_recommendation(
    query_dict=query_dict,
    movies_data=movies_df,
    tfidf_vectorizer=tfidf_vectorizer,
    bert_model=bert_model,
    combined_features=combined_features,
    top_n=5,
)

# Hiển thị kết quả
print("Recommended movies based on conditions:")
for title, score in recommendations:
    print(f"{title}: {score:.2f}")


Recommended movies based on conditions:
Commitment to Life: 0.15
Asteroid City: 0.14
California Typewriter: 0.13
Greyhound: 0.13
Everything Is Copy: 0.13


In [37]:
# Tạo query với nhiều điều kiện
query_dict = {
   "Genres": "Drama, War"
}


# Gợi ý phim dựa trên các điều kiện
recommendations = multi_condition_recommendation(
    query_dict=query_dict,
    movies_data=movies_df,
    tfidf_vectorizer=tfidf_vectorizer,
    bert_model=bert_model,
    combined_features=combined_features,
    top_n=5,
)

# Hiển thị kết quả
print("Recommended movies based on conditions:")
for title, score in recommendations:
    print(f"{title}: {score:.2f}")


Recommended movies based on conditions:
The War Around Us: 0.22
Signs of War: 0.20
Hondros: 0.18
The November War: 0.18
The Volunteers: 0.17


In [31]:
import pickle

# Lưu mô hình TF-IDF
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

# Lưu mô hình BERT
with open('bert_model.pkl', 'wb') as f:
    pickle.dump(bert_model, f)

# Lưu ma trận các đặc trưng kết hợp (TF-IDF + BERT)
with open('combined_features.pkl', 'wb') as f:
    pickle.dump(combined_features, f)

# Lưu dataset (dữ liệu phim)
with open('movies_data.pkl', 'wb') as f:
    pickle.dump(movies_df, f)